In [1]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved to /home/hilhag/.huggingface/token
Login successful


In [2]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sklearn.utils import shuffle

In [3]:
import pandas as pd

In [25]:
odf = df

In [26]:
d = []
test_file = open("emo.txt")
t = test_file.readlines()
i = 0
for line in t:
    i += 1
    #print(i)
    l = line.split(",")
    text = l[0].strip()
    sent = l[1].strip()
    sent = int(sent)
    d.append([text, sent])

In [28]:
df = pd.DataFrame(d)

In [48]:
df = shuffle(df)

In [49]:
df_2 = df.head(69)
df_3 = df.tail(69)

In [50]:
headers =  ["text", "label"]
df_2.columns = headers
df_3.columns = headers

In [42]:
from datasets import Dataset

In [51]:
training_set = Dataset.from_pandas(df_3)
test_set = Dataset.from_pandas(df_2)

In [52]:
#dataset = load_dataset("SetFit/SentEval-CR")

In [53]:
# Load SetFit model from Hub
#model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
model = SetFitModel.from_pretrained("KBLab/sentence-bert-swedish-cased")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=training_set,
    eval_dataset=test_set,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [54]:
# Train and evaluate!
trainer.train()
metrics = trainer.evaluate()

***** Running training *****
  Num examples = 2760
  Num epochs = 1
  Total optimization steps = 173
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/173 [00:00<?, ?it/s]

***** Running evaluation *****


In [55]:
metrics

{'accuracy': 0.6956521739130435}

In [14]:
trainer.push_to_hub("sent-setfit-test")

/home/hilhag/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'push_to_hub': pass repo_path_or_name='sent-setfit-test', repo_url=None, commit_message='Add SetFit model', organization=None, private=None, api_endpoint=None, token=None, git_user=None, git_email=None, config=None, skip_lfs_files=False as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  warnings.warn(
/home/hilhag/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'push_to_hub': repo_path_or_name. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


KeyboardInterrupt: 